In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# # Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("../weather_data/cities.csv")
city_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.1927,168.8643,43.48,99,31,2.51,NZ,2022-02-13 16:27:18
1,Nikolskoye,59.7035,30.7861,34.88,85,100,12.46,RU,2022-02-13 16:27:19
2,Ancud,-41.8697,-73.8203,82.22,72,1,6.76,CL,2022-02-13 16:27:20
3,Saint-Philippe,-21.3585,55.7679,80.19,86,53,8.75,RE,2022-02-13 16:27:21
4,Kapaa,22.0752,-159.3190,70.12,85,0,12.66,US,2022-02-13 16:27:21


In [3]:
city_data_df = city_data_df.loc[city_data_df["Max Temp"]>=0]

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# 5. Add the heatmap layer.
fig.add_layer(heatmap_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
clouds = city_data_df["Cloudiness"]

In [13]:
wind = city_data_df["Wind Speed"]

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 75


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.1927,168.8643,43.48,99,31,2.51,NZ,2022-02-13 16:27:18
1,Nikolskoye,59.7035,30.7861,34.88,85,100,12.46,RU,2022-02-13 16:27:19
4,Kapaa,22.0752,-159.3190,70.12,85,0,12.66,US,2022-02-13 16:27:21
6,Punta Arenas,-53.1500,-70.9167,50.11,57,20,57.53,CL,2022-02-13 16:27:23
7,Avarua,-21.2078,-159.7750,73.45,100,20,2.30,CK,2022-02-13 16:27:24
8,Port Elizabeth,-33.9180,25.5701,71.89,78,100,9.22,ZA,2022-02-13 16:27:25
10,Muros,42.7762,-9.0603,55.09,84,81,19.13,ES,2022-02-13 16:27:26
11,Mwinilunga,-11.7358,24.4293,69.28,82,96,3.56,ZM,2022-02-13 16:27:27
12,Albany,42.6001,-73.9662,26.55,64,100,1.01,US,2022-02-13 16:27:28
13,Ushuaia,-54.8000,-68.3000,47.86,71,75,47.18,AR,2022-02-13 16:27:29


In [18]:
preferred_cities_df.count()

City          311
Lat           311
Lng           311
Max Temp      311
Humidity      311
Cloudiness    311
Wind Speed    311
Country       310
Date          311
dtype: int64

In [19]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mataura,NZ,43.48,-46.1927,168.8643,
1,Nikolskoye,RU,34.88,59.7035,30.7861,
4,Kapaa,US,70.12,22.0752,-159.3190,
6,Punta Arenas,CL,50.11,-53.1500,-70.9167,
7,Avarua,CK,73.45,-21.2078,-159.7750,
8,Port Elizabeth,ZA,71.89,-33.9180,25.5701,
10,Muros,ES,55.09,42.7762,-9.0603,
11,Mwinilunga,ZM,69.28,-11.7358,24.4293,
12,Albany,US,26.55,42.6001,-73.9662,
13,Ushuaia,AR,47.86,-54.8000,-68.3000,


In [21]:
# Iterate through the DataFrame.
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name. 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer[(locations, info_box_content=hotel_info)]
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

SyntaxError: invalid syntax (Temp/ipykernel_25960/3167471083.py, line 7)